In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import time 
import pandas as pd
import json

In [21]:
hh_main_link = 'https://hh.ru/'
sj_main_link = 'https://www.superjob.ru/'
header = {'Accept': '*/*',
          'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0'}
proxi = {'http': ''}
search_str = 'Data+science'
first_page = 0

In [22]:
def hh(hh_main_link, search_str, first_page):
    data = list()
    html = requests.get(f'{hh_main_link}search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text={search_str}&page={first_page}', headers=header, proxies=proxi).text
    parsed_html = bs(html, 'lxml')
    #Находим номер последней страницы 
    last_page = int(parsed_html.find_all(attrs={'class':'bloko-button HH-Pager-Control'})[-1]['data-page'])

    
    for i in range(last_page+1): 
        vacancy_block = parsed_html.find('div',{'class':'vacancy-serp'} )
        vacancy_list = vacancy_block.findChildren(recursive=False)
        for vacancy in vacancy_list:
            vacancy_dict={}
            a = vacancy.find('a',{'class':'bloko-link HH-LinkModifier'})
            if a != None:
                vacancy_dict['site'] = 'HH'
                vacancy_dict['name'] = a.getText()
                vacancy_dict['link'] = a['href']
                compensation = vacancy.find('div', {'class': 'vacancy-serp-item__sidebar'}).text.replace('\xa0', ' ')
                if not compensation:
                    vacancy_dict['compensation_min'] = 'По договоренности'
                    vacancy_dict['compensation_max'] = ' '
                else:
                    compensation = compensation.replace('\xa0', ' ').split('-')
                    compensation_min=compensation[0]
                    vacancy_dict['compensation_min'] =compensation_min
                    if len(compensation)>1:
                        compensation_max=compensation[1]
                        vacancy_dict['compensation_max'] = compensation_max
                    else:
                        compensation_max= ' '
                        vacancy_dict['compensation_max'] = compensation_max
                    
                data.append(vacancy_dict)
        
        first_page +=1
        time.sleep(1)
        html = requests.get(f'{hh_main_link}search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text={search_str}&page={first_page}', headers=header, proxies=proxi).text
        parsed_html = bs(html, 'lxml')
    return data

In [23]:
result = hh(hh_main_link, search_str,first_page)

In [24]:
with open("Lesson2_result.json", "w", encoding="utf-8") as file:
    json.dump(result, file)

In [25]:
df = pd.DataFrame(result)
df

,site,name,link,compensation_min,compensation_max
0,HH,Fullstack Python Developer,https://spb.hh.ru/vacancy/36222549?query=Data%...,1 700,2 000 USD
1,HH,"Специалист / data scientist (big data, прогнос...",https://spb.hh.ru/vacancy/35895583?query=Data%...,от 100 000 руб.,
2,HH,Data Science / Machine Learning,https://spb.hh.ru/vacancy/35884655?query=Data%...,По договоренности,
3,HH,Руководитель отдела Data science,https://spb.hh.ru/vacancy/35896174?query=Data%...,По договоренности,
4,HH,DATA-аналитик,https://spb.hh.ru/vacancy/35815966?query=Data%...,По договоренности,
...,...,...,...,...,...
402,HH,Старший С++ разработчик,https://spb.hh.ru/vacancy/35893444?query=Data%...,По договоренности,
403,HH,"Руководитель направления клиентской аналитики,...",https://spb.hh.ru/vacancy/35993000?query=Data%...,По договоренности,
404,HH,Разработчик ETL,https://spb.hh.ru/vacancy/35886021?query=Data%...,По договоренности,
405,HH,Full Stack Developer,https://spb.hh.ru/vacancy/35985747?query=Data%...,По договоренности,
